# Scratchwork: experimenting with "words"

In [1]:
import finite_algebras as alg

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

In [3]:
ex = alg.Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  16 example algebras are available.
  Use "Examples[INDEX]" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-groups.m

## Klein-4 Group

In [4]:
v4 = ex[7]
v4

Group(
'V4',
'Klein-4 group',
['e', 'h', 'v', 'r'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

In [5]:
v4.about()


** Group **
Name: V4
Instance ID: 140409474175184
Description: Klein-4 group
Order: 4
Identity: e
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       h       h       2
      2       v       v       2
      3       r       r       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


### Finite Presentation

$V_4 = \langle h, v | h^2 = v^2 = (hv)^2 = e \rangle$

Generators and Relators:

In [6]:
gens = ['h', 'v']

In [7]:
rels = [('h_h', ''), ('v_v', ''), ('h_v_h_v', '')]
rels

[('h_h', ''), ('v_v', ''), ('h_v_h_v', '')]

In [8]:
import itertools as it

def cartesian_product(elems):
    '''EXAMPLE: cartesian_product(['h', 'v'])
    ==> ['h_h', 'h_v', 'v_h', 'v_v']'''
    return map(lambda x: '_'.join(x),
               it.product(elems, elems))

In [9]:
elems = list(cartesian_product(['h', 'v']))
elems

['h_h', 'h_v', 'v_h', 'v_v']

In [10]:
def remove_items_from_list(lst, items):
    '''Return the list that results from removing all items in
    items from lst.'''
    if items == []:
        return lst
    else:
        return remove_items_from_list([x for x in lst if x != items[0]],
                                      items[1:])

In [11]:
remove_items_from_list(['a', 'b', 'c', 'd', 'e'], ['a', 'c'])

['b', 'd', 'e']

In [12]:
def reduce_word_by_rel(word, rel):
    '''Reduce word using a single relator, rel; return the result.'''
    if rel[1] == '':
        rel_ = rel[0] + '_'
        _rel = '_' + rel[0]
        if word == rel[0]:
            return rel[1]
        elif rel_ in word:
            return word.replace(rel_, rel[1])
        elif _rel in word:
            return word.replace(_rel, rel[1])
        else:
            return word
    else:
        return word.replace(rel[0], rel[1])

In [13]:
reduce_word_by_rel('v_h_h_v', ('h_h', ''))

'v_v'

In [14]:
reduce_word_by_rel('v_h_h_v', ('v_h', ''))

'h_v'

In [15]:
reduce_word_by_rel('v_h_h_v', ('h_v', ''))

'v_h'

In [16]:
reduce_word_by_rel('v_h_h_v', ('foo', ''))

'v_h_h_v'

In [17]:
reduce_word_by_rel('v_h_h_v', ('h_h_v', 'v'))

'v_v'

In [18]:
def reduce_word_once(word, rels):
    '''Use a list of relators, rels, to reduce a word,
    but only do one pass through rels'''
    if rels == []:
        return word
    elif rels[0][0] in word:
        return reduce_word_by_rel(word, rels[0])
    else:
        return reduce_word_once(word, rels[1:])

In [19]:
rels

[('h_h', ''), ('v_v', ''), ('h_v_h_v', '')]

In [20]:
reduce_word_once('h_v_v_h', rels)

'h_h'

In [21]:
reduce_word_once('v_v', rels)

''

In [22]:
reduce_word_once('h_v_h_X', [('foo', 'bar'), ('h_v_h', 'v')])

'v_X'

In [23]:
def reduce_word_recursively(word, rels):
    '''Use a list of relators, rels, to reduce a word,
    and make as many passes through rels as necessary to
    reduce the word as far as possible.'''
    def reduce_word_aux(old, new):
        if new == old:
            return new
        else:
            return reduce_word_aux(new, reduce_word_once(new, rels))
    return reduce_word_aux('', word)

In [24]:
rels

[('h_h', ''), ('v_v', ''), ('h_v_h_v', '')]

In [25]:
reduce_word_recursively('X_h_v_h_h_h_v', rels)

'X'

In [26]:
foo = [('h_h', 'X'), ('v_v', 'Y'), ('X_h_v', 'Z')]

In [27]:
reduce_word_recursively('X_h_v_h_h_h_v', foo)

'Z_Z'

In [28]:
# def multiply_words_OLD(word1, word2, rels):
#     '''Multiply two words and return the reduced result.'''
#     prod0 = word1 + '_' + word2
#     prod1 = reduce_word_recursively(prod0, rels)
#     return prod1

In [29]:
def multiply_words(word1, word2, rels):
    '''Multiply two words and return the reduced result.'''
    if word1 == '':
        if word2 == '':
            return ''
        else:
            return reduce_word_recursively(word2, rels)
    else:
        if word2 == '':
            return reduce_word_recursively(word1, rels)
        else:
            return reduce_word_recursively(word1 + '_' + word2, rels)

In [30]:
gens

['h', 'v']

In [31]:
rels

[('h_h', ''), ('v_v', ''), ('h_v_h_v', '')]

In [32]:
for g in gens:
    for r in rels:
        a = multiply_words(g, r[0], rels)
        b = multiply_words(r[0], g, rels)
        if a != b:
            if len(a) >= len(b):
                print(f"{a} == {b}")
            else:
                print(f"{b} == {a}")

v_h_v == h
h_v_h == v


In [34]:
def reduce_words(words, rels):
    '''Reduce a list of words using a list of relators.'''
    return list(set(map(lambda x: reduce_word_recursively(x, rels),
                        words)))

In [35]:
cartprod = list(cartesian_product(gens))
cartprod

['h_h', 'h_v', 'v_h', 'v_v']

Remove relations that occur in whole form in the list:

In [36]:
new_items = remove_items_from_list(cartprod, rels)
new_items

['h_h', 'h_v', 'v_h', 'v_v']

Add new items to the generators to create a new list of element candidates:

In [37]:
elems = gens + new_items
elems

['h', 'v', 'h_h', 'h_v', 'v_h', 'v_v']

Now, cross-multiply the new list of element candidates:

In [38]:
foo = list(cartesian_product(elems))
foo

['h_h',
 'h_v',
 'h_h_h',
 'h_h_v',
 'h_v_h',
 'h_v_v',
 'v_h',
 'v_v',
 'v_h_h',
 'v_h_v',
 'v_v_h',
 'v_v_v',
 'h_h_h',
 'h_h_v',
 'h_h_h_h',
 'h_h_h_v',
 'h_h_v_h',
 'h_h_v_v',
 'h_v_h',
 'h_v_v',
 'h_v_h_h',
 'h_v_h_v',
 'h_v_v_h',
 'h_v_v_v',
 'v_h_h',
 'v_h_v',
 'v_h_h_h',
 'v_h_h_v',
 'v_h_v_h',
 'v_h_v_v',
 'v_v_h',
 'v_v_v',
 'v_v_h_h',
 'v_v_h_v',
 'v_v_v_h',
 'v_v_v_v']

In [39]:
foo = remove_items_from_list(foo, elems + rels)
foo

['h_h_h',
 'h_h_v',
 'h_v_h',
 'h_v_v',
 'v_h_h',
 'v_h_v',
 'v_v_h',
 'v_v_v',
 'h_h_h',
 'h_h_v',
 'h_h_h_h',
 'h_h_h_v',
 'h_h_v_h',
 'h_h_v_v',
 'h_v_h',
 'h_v_v',
 'h_v_h_h',
 'h_v_h_v',
 'h_v_v_h',
 'h_v_v_v',
 'v_h_h',
 'v_h_v',
 'v_h_h_h',
 'v_h_h_v',
 'v_h_v_h',
 'v_h_v_v',
 'v_v_h',
 'v_v_v',
 'v_v_h_h',
 'v_v_h_v',
 'v_v_v_h',
 'v_v_v_v']

In [40]:
foo2 = reduce_words(foo, rels)
foo2

['', 'v_h_v_h', 'h', 'v_h_v', 'h_v', 'v_h', 'h_v_h', 'v']

In [41]:
foo3 = remove_items_from_list(foo2, elems + rels)
foo3

['', 'v_h_v_h', 'v_h_v', 'h_v_h']

In [42]:
multiply_words('h', 'v', rels)

'h_v'

In [43]:
gens

['h', 'v']

In [44]:
pairs = list(it.product(gens, gens))
print(pairs)

fubar = list(set(map(lambda x: multiply_words(*x, rels), pairs)))
fubar.remove('')
print(fubar)

[('h', 'h'), ('h', 'v'), ('v', 'h'), ('v', 'v')]
['v_h', 'h_v']


In [45]:
elems = gens + fubar

pairs = list(it.product(elems, elems))
print(pairs)

fubar = list(set(map(lambda x: multiply_words(*x, rels), pairs)))
fubar.remove('')
print(fubar)

[('h', 'h'), ('h', 'v'), ('h', 'v_h'), ('h', 'h_v'), ('v', 'h'), ('v', 'v'), ('v', 'v_h'), ('v', 'h_v'), ('v_h', 'h'), ('v_h', 'v'), ('v_h', 'v_h'), ('v_h', 'h_v'), ('h_v', 'h'), ('h_v', 'v'), ('h_v', 'v_h'), ('h_v', 'h_v')]
['v_h_v_h', 'h', 'h_v', 'v_h_v', 'v_h', 'h_v_h', 'v']
